In [5]:
import pandas as pd
df = pd.read_excel("정보성키워드_20240229.xlsx")


In [1]:
from src.search_keywords.naver_keywords_collector import 연관키워드수집

현재 Chrome 버전: 134.0.6998.36
ChromeDriver 설치 완료: c:\Users\PC_1M\anaconda3\envs\py312\Lib\site-packages\chromedriver_autoinstaller\134\chromedriver.exe
ChromeDriver 설치 완료: c:\Users\PC_1M\anaconda3\envs\py312\Lib\site-packages\chromedriver_autoinstaller\134\chromedriver.exe
Chrome WebDriver 생성 완료


In [2]:
연관키워드수집("골밀도")

,연관키워드,월간검색수_PC,월간검색수_모바일,월평균클릭수_PC,월평균클릭수_모바일,월평균클릭률_PC,월평균클릭률_모바일,경쟁정도,월평균노출광고수
0,골밀도,530,1800,0.0,2.5,0.00,0.15,높음,7
1,관절영양제,5940,31500,21.5,601.5,0.40,2.06,높음,15
2,콘드로이친,22200,184000,130.5,6133.0,0.64,3.59,높음,15
3,손가락관절통증,1620,15700,1.3,99.1,0.09,0.68,높음,15
4,콘드로이친1200,6100,83400,69.8,4264.0,1.23,5.54,높음,15
5,무릎관절영양제,530,3630,3.9,144.6,0.77,4.24,높음,15
6,콘드로이친추천,1690,14000,24.1,814.1,1.53,6.31,높음,15
7,관절약추천,170,1340,1.2,30.0,0.71,2.43,높음,15
8,관절염영양제,340,3280,2.3,60.1,0.71,1.98,높음,15
9,관절,1130,6030,1.0,26.5,0.10,0.47,높음,7


In [3]:
from src.generator.text.title_generator import generate_title
topic = generate_title("관절연골영양제")



C:\Users\PC_1M\AppData\Local\Temp\ipykernel_29276\722588490.py:1: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from src.generator.text.title_generator import generate_title
c:\Users\PC_1M\anaconda3\envs\py312\Lib\site-packages\langchain_openai\chat_models\base.py:1377: UserWarning: Received a Pydantic BaseModel V1 schema. This is not supported by method="json_schema". Please use method="function_calling" or specify schema via JSON Schema or Pydantic V2 BaseModel. Overriding to method="function

In [5]:
topic

'관절연골영양제의 효과와 선택 가이드'

In [1]:
from langgraph.checkpoint.memory import MemorySaver
from src.graph import builder
from langgraph.types import Command
memory = MemorySaver()
graph = builder.compile(checkpointer=memory)

import uuid 
thread = {"configurable": {"thread_id": str(uuid.uuid4()),
                           "search_api": "pubmed", # tavily, perplexity, exa, arxiv, pubmed, linkup
                           "planner_provider": "openai",# 
                           "planner_model": "gpt-4o-mini",
                           "writer_provider": "openai",
                           "writer_model": "gpt-4o-mini",
                           "max_search_depth": 3,
                           }}

topic = '관절연골영양제의 효과와 선택 가이드'
async for event in graph.astream({"topic":topic,}, thread, stream_mode="updates"):
    print(event)
    print("\n")
    
        
events = []
async for event in graph.astream(Command(resume=True), thread, stream_mode="updates"):
    events.append(event)
    print(event)
    print("\n")

HTTPStatusError: Client error '403 Forbidden' for url 'https://api.tavily.com/search'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/403

In [1]:
from src.keyword_classifier import naver_search
data = naver_search(query='관절연골영양제의 효과와 선택 가이드' , search_type="kin")

C:\Users\PC_1M\AppData\Local\Temp\ipykernel_5396\918522599.py:1: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from src.keyword_classifier import naver_search
c:\Users\PC_1M\anaconda3\envs\py312\Lib\site-packages\langchain_openai\chat_models\base.py:1377: UserWarning: Received a Pydantic BaseModel V1 schema. This is not supported by method="json_schema". Please use method="function_calling" or specify schema via JSON Schema or Pydantic V2 BaseModel. Overriding to method="function_calling".
  wa

In [2]:
data

{'lastBuildDate': 'Wed, 12 Mar 2025 21:22:42 +0900',
 'total': 5967,
 'start': 1,
 'display': 10,
 'items': [{'title': '<b>관절연골영양제</b> 성분 봐주세요(성동구 <b>관절</b>염)',
   'link': 'https://kin.naver.com/qna/detail.naver?d1id=7&dirId=71103&docId=481262316&qb=6rSA7KCI7Jew6rOo7JiB7JaR7KCc7J2YIO2aqOqzvOyZgCDshKDtg50g6rCA7J2065Oc&enc=utf8',
   'description': '... 섭취하려는데요 <b>관절연골영양제</b> <b>관절</b>스타 성분 봐주세요 # 문의 전화... <b>영양제</b> <b>선택</b> 시 가장 중요한게 성분과 함량인데, <b>관절</b>스타는 기능성 원료가 총... 바라며, <b>영양제</b>는 3개월 이상 꾸준히 복용하셔야 <b>효과</b>를 볼 수 있다는 점... '},
  {'title': '<b>관절 연골 영양제</b> <b>선택</b>의 기준은? 경험자 의견 부탁드립',
   'link': 'https://kin.naver.com/qna/detail.naver?d1id=8&dirId=8020306&docId=474362508&qb=6rSA7KCI7Jew6rOo7JiB7JaR7KCc7J2YIO2aqOqzvOyZgCDshKDtg50g6rCA7J2065Oc&enc=utf8',
   'description': '... <b>관절 연골</b>이 많이 약해졌다는 진단을 받아서 <b>관절 연골 영양제</b> 를 찾고 있는데, 종류가 너무 많아서 <b>선택</b>이 어렵네요. <b>관절 연골 영양제</b> 중에 <b>효과</b> 본 제품이 있으시면 추천 부탁드려요. 글루코사민, 콘드로이친 말고도 괜찮은 게... '},
  {'title': '무릎<b>연골</b>에 <b>효과</b> 확실한 <b>

In [4]:
<h1 style="font-size: 2.2rem; font-weight: 700; color: #4a148c; margin-top: 1.5em; margin-bottom: 0.8em; line-height: 1.2; border-bottom: 3px solid #7b1fa2; padding-bottom: 0.3em; font-family: 'Noto Sans KR', sans-serif;">
관절연골영양제의 효과와 선택 가이드
</h1>

<!-- 후킹 문구 -->
<p style="font-size: 1.1rem; line-height: 1.8; margin-bottom: 1.2em; color: #6a1b9a; font-weight: 500; font-family: 'Noto Sans KR', sans-serif;">
무릎이 뻐근하고 걸을 때마다 찌릿한 통증이 느껴지시나요? 관절 건강을 지키기 위한 가장 효과적인 방법, 제대로 알고 계신가요?
</p>

<!-- 인삿말 -->
<p style="font-size: 1rem; line-height: 1.8; margin-bottom: 1.2em; color: #333; font-family: 'Noto Sans KR', sans-serif;">
안녕하세요, 여러분! 나이가 들수록 관절이 약해지고, 특히 연골 손상은 삶의 질에 큰 영향을 미칩니다. 계단을 오르내릴 때나 장시간 서 있을 때 무릎이 아픈 경험, 한 번쯤 해보셨을 텐데요. 저 역시 몇 년 전부터 무릎이 뻐근해지는 걸 느끼면서 관절 건강의 중요성을 깨닫게 됐어요.  
이런 고민을 가진 분들을 위해 오늘은 관절연골영양제의 효과와 올바른 선택법에 대해 자세히 알아보려 합니다. 시중에 다양한 제품이 넘쳐나지만, 내 몸에 맞는 제품을 고르는 것이 중요합니다. 그럼 지금부터 건강한 관절을 위한 필수 정보를 살펴볼까요?
</p>

<!-- 목차 -->
<div style="background: linear-gradient(135deg, #f5f0ff, #f0e6ff); border-radius: 8px; padding: 20px; margin: 20px 0; box-shadow: 0 4px 10px rgba(106, 27, 154, 0.08); font-family: 'Noto Sans KR', sans-serif; border-left: 4px solid #9c27b0;">
  
  <!-- 제목 영역 -->
  <div style="margin-bottom: 15px;">
    <h3 style="font-weight: 700; color: #4a148c; font-size: 18px; margin: 0;">목차</h3>
  </div>
  
  <!-- 목차 내용 -->
  <div style="display: flex; flex-direction: column; gap: 10px;">
    <a href="#joint-health" style="color: #6a1b9a; text-decoration: none; font-weight: 500;">1. 관절과 연골 건강의 중요성</a>
    <a href="#supplement-effect" style="color: #6a1b9a; text-decoration: none; font-weight: 500;">2. 관절연골영양제의 주요 성분과 효과</a>
    <a href="#choose-right" style="color: #6a1b9a; text-decoration: none; font-weight: 500;">3. 좋은 관절영양제를 고르는 기준</a>
    <a href="#best-products" style="color: #6a1b9a; text-decoration: none; font-weight: 500;">4. 인기 있는 관절연골영양제 추천</a>
    <a href="#how-to-take" style="color: #6a1b9a; text-decoration: none; font-weight: 500;">5. 효과적으로 섭취하는 방법</a>
    <a href="#lifestyle-tips" style="color: #6a1b9a; text-decoration: none; font-weight: 500;">6. 관절 건강을 위한 생활 습관</a>
  </div>
</div>

<!-- 첫 번째 섹션 -->
<h2 id="joint-health" style="font-size: 1.8rem; font-weight: 700; color: #4a148c; margin-top: 1.5em; margin-bottom: 0.8em; line-height: 1.2; border-left: 5px solid #9c27b0; padding-left: 0.8em; font-family: 'Noto Sans KR', sans-serif;">
관절과 연골 건강의 중요성
</h2>

<p style="font-size: 1rem; line-height: 1.8; margin-bottom: 1.2em; color: #333; font-family: 'Noto Sans KR', sans-serif;">
우리 몸의 관절과 연골은 움직임을 부드럽게 해주는 중요한 역할을 합니다. 특히 무릎, 엉덩이, 손목 등의 관절은 나이가 들면서 점점 퇴행성 변화를 겪게 됩니다.  
연골은 뼈와 뼈 사이에서 쿠션 역할을 하며 충격을 흡수하는데, 연골이 닳거나 손상되면 관절염이 발생할 가능성이 높아집니다. 이 때문에 적절한 영양 공급과 관리가 필수적입니다.
</p>

<p style="font-size: 1rem; line-height: 1.8; margin-bottom: 1.2em; color: #333; font-family: 'Noto Sans KR', sans-serif;">
특히 40대 이후부터는 연골이 자연스럽게 마모되기 시작하며, 이를 방치하면 통증과 불편함이 심해질 수 있습니다. 따라서 조기 예방과 꾸준한 관리가 중요합니다.
</p>

<!-- 두 번째 섹션 (테이블 포함) -->
<h2 id="supplement-effect" style="font-size: 1.8rem; font-weight: 700; color: #4a148c; margin-top: 1.5em; margin-bottom: 0.8em; line-height: 1.2; border-left: 5px solid #9c27b0; padding-left: 0.8em; font-family: 'Noto Sans KR', sans-serif;">
관절연골영양제의 주요 성분과 효과
</h2>

<p style="font-size: 1rem; line-height: 1.8; margin-bottom: 1.2em; color: #333; font-family: 'Noto Sans KR', sans-serif;">
시중에 나와 있는 다양한 관절연골영양제에는 여러 가지 핵심 성분이 포함되어 있습니다. 각 성분마다 효능이 다르므로 자신의 관절 상태에 맞는 성분을 선택하는 것이 중요합니다.
</p>

<!-- 성분 및 효과 테이블 -->
<table style="width: 100%; border-collapse: collapse; margin: 1.5em 0; font-family: 'Noto Sans KR', sans-serif;">
  <thead>
    <tr>
      <th style="border: 1px solid #e1bee7; padding: 0.5em; text-align: left; background-color: #9c27b0; color: white; font-weight: 700;">성분</th>
      <th style="border: 1px solid #e1bee7; padding: 0.5em; text-align: left; background-color: #9c27b0; color: white; font-weight: 700;">효과</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td style="border: 1px solid #e1bee7; padding: 0.5em; text-align: left;">글루코사민</td>
      <td style="border: 1px solid #e1bee7; padding: 0.5em; text-align: left;">연골 형성 촉진, 관절염 예방</td>
    </tr>
    <tr style="background-color: #f3e5f5;">
      <td style="border: 1px solid #e1bee7; padding: 0.5em; text-align: left;">콘드로이틴</td>
      <td style="border: 1px solid #e1bee7; padding: 0.5em; text-align: left;">연골 보호, 염증 완화</td>
    </tr>
    <tr>
      <td style="border: 1px solid #e1bee7; padding: 0.5em; text-align: left;">MSM(메틸설포닐메탄)</td>
      <td style="border: 1px solid #e1bee7; padding: 0.5em; text-align: left;">관절 통증 완화, 항염 작용</td>
    </tr>
    <tr style="background-color: #f3e5f5;">
      <td style="border: 1px solid #e1bee7; padding: 0.5em; text-align: left;">콜라겐</td>
      <td style="border: 1px solid #e1bee7; padding: 0.5em; text-align: left;">연골 탄력 강화, 피부 건강에도 도움</td>
    </tr>
  </tbody>
</table>

<!-- 세 번째 섹션 (리스트 포함) -->
<h2 id="choose-right" style="font-size: 1.8rem; font-weight: 700; color: #4a148c; margin-top: 1.5em; margin-bottom: 0.8em; line-height: 1.2; border-left: 5px solid #9c27b0; padding-left: 0.8em; font-family: 'Noto Sans KR', sans-serif;">
좋은 관절영양제를 고르는 기준
</h2>

<p style="font-size: 1rem; line-height: 1.8; margin-bottom: 1.2em; color: #333; font-family: 'Noto Sans KR', sans-serif;">
수많은 관절영양제 중에서 어떤 제품이 좋은 제품인지 고민이 되시나요? 다음과 같은 기준을 참고하면 더 쉽게 선택할 수 있습니다.
</p>

<!-- 선택 기준 리스트 -->
<ul style="margin-bottom: 1.2em; padding-left: 2em; list-style-type: none; color: #333; font-family: 'Noto Sans KR', sans-serif;">
  <li style="margin-bottom: 0.5em; position: relative;">
    <span style="display: inline-block; width: 8px; height: 8px; border-radius: 50%; background-color: #9c27b0; margin-right: 10px; position: absolute; left: -20px; top: 8px;"></span>
    <strong>주요 성분 함량</strong> – 글루코사민, 콘드로이틴, MSM 등의 함량이 충분한지 확인하세요.
  </li>
  <li style="margin-bottom: 0.5em; position: relative;">
    <span style="display: inline-block; width: 8px; height: 8px; border-radius: 50%; background-color: #9c27b0; margin-right: 10px; position: absolute; left: -20px; top: 8px;"></span>
    <strong>첨가물 확인</strong> – 불필요한 합성 첨가물이 들어있지 않은지 체크하세요.
  </li>
  <li style="margin-bottom: 0.5em; position: relative;">
    <span style="display: inline-block; width: 8px; height: 8px; border-radius: 50%; background-color: #9c27b0; margin-right: 10px; position: absolute; left: -20px; top: 8px;"></span>
    <strong>흡수율</strong> – 체내 흡수율이 높은 형태(예: 액상, 가수분해된 콜라겐 등)를 선택하세요.
  </li>
  <li style="margin-bottom: 0.5em; position: relative;">
    <span style="display: inline-block; width: 8px; height: 8px; border-radius: 50%; background-color: #9c27b0; margin-right: 10px; position: absolute; left: -20px; top: 8px;"></span>
    <strong>신뢰할 수 있는 브랜드</strong> – 인증받은 제조업체의 제품인지 확인하세요.
  </li>
</ul>

<!-- 네 번째 섹션 -->
<h2 id="best-products" style="font-size: 1.8rem; font-weight: 700; color: #4a148c; margin-top: 1.5em; margin-bottom: 0.8em; line-height: 1.2; border-left: 5px solid #9c27b0; padding-left: 0.8em; font-family: 'Noto Sans KR', sans-serif;">
인기 있는 관절연골영양제 추천
</h2>

<p style="font-size: 1rem; line-height: 1.8; margin-bottom: 1.2em; color: #333; font-family: 'Noto Sans KR', sans-serif;">
시장에는 다양한 관절연골영양제가 출시되어 있지만, 효과적인 제품을 고르는 것이 중요합니다. 아래는 소비자 만족도가 높은 대표적인 제품들입니다.
</p>

<!-- 인기 제품 테이블 -->
<table style="width: 100%; border-collapse: collapse; margin: 1.5em 0; font-family: 'Noto Sans KR', sans-serif;">
  <thead>
    <tr>
      <th style="border: 1px solid #e1bee7; padding: 0.5em; text-align: left; background-color: #9c27b0; color: white; font-weight: 700;">제품명</th>
      <th style="border: 1px solid #e1bee7; padding: 0.5em; text-align: left; background-color: #9c27b0; color: white; font-weight: 700;">주요 성분</th>
      <th style="border: 1px solid #e1bee7; padding: 0.5em; text-align: left; background-color: #9c27b0; color: white; font-weight: 700;">특징</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td style="border: 1px solid #e1bee7; padding: 0.5em; text-align: left;">○○ 글루코사민 플러스</td>
      <td style="border: 1px solid #e1bee7; padding: 0.5em; text-align: left;">글루코사민, 콘드로이틴</td>
      <td style="border: 1px solid #e1bee7; padding: 0.5em; text-align: left;">연골 형성 촉진, 관절 보호</td>
    </tr>
    <tr style="background-color: #f3e5f5;">
      <td style="border: 1px solid #e1bee7; padding: 0.5em; text-align: left;">△△ MSM 파우더</td>
      <td style="border: 1px solid #e1bee7; padding: 0.5em; text-align: left;">MSM, 비타민C</td>
      <td style="border: 1px solid #e1bee7; padding: 0.5em; text-align: left;">항염 작용, 통증 완화</td>
    </tr>
    <tr>
      <td style="border: 1px solid #e1bee7; padding: 0.5em; text-align: left;">☆☆ 콜라겐 보충제</td>
      <td style="border: 1px solid #e1bee7; padding: 0.5em; text-align: left;">콜라겐 펩타이드, 히알루론산</td>
      <td style="border: 1px solid #e1bee7; padding: 0.5em; text-align: left;">연골 탄력 증진, 피부 건강 개선</td>
    </tr>
  </tbody>
</table>

<!-- 다섯 번째 섹션 -->
<h2 id="how-to-take" style="font-size: 1.8rem; font-weight: 700; color: #4a148c; margin-top: 1.5em; margin-bottom: 0.8em; line-height: 1.2; border-left: 5px solid #9c27b0; padding-left: 0.8em; font-family: 'Noto Sans KR', sans-serif;">
효과적으로 섭취하는 방법
</h2>

<p style="font-size: 1rem; line-height: 1.8; margin-bottom: 1.2em; color: #333; font-family: 'Noto Sans KR', sans-serif;">
관절연골영양제는 단순히 먹는 것만으로 효과를 보는 것이 아니라, 올바른 복용법을 지켜야 효과를 극대화할 수 있습니다.
</p>

<ul style="margin-bottom: 1.2em; padding-left: 2em; list-style-type: none; color: #333; font-family: 'Noto Sans KR', sans-serif;">
  <li style="margin-bottom: 0.5em; position: relative;">
    <span style="display: inline-block; width: 8px; height: 8px; border-radius: 50%; background-color: #9c27b0; margin-right: 10px; position: absolute; left: -20px; top: 8px;"></span>
    <strong>공복보다 식사 후 섭취</strong> – 위장 부담을 줄이기 위해 식사와 함께 복용하는 것이 좋습니다.
  </li>
  <li style="margin-bottom: 0.5em; position: relative;">
    <span style="display: inline-block; width: 8px; height: 8px; border-radius: 50%; background-color: #9c27b0; margin-right: 10px; position: absolute; left: -20px; top: 8px;"></span>
    <strong>꾸준한 섭취</strong> – 최소 3개월 이상 지속적으로 복용해야 효과를 볼 수 있습니다.
  </li>
  <li style="margin-bottom: 0.5em; position: relative;">
    <span style="display: inline-block; width: 8px; height: 8px; border-radius: 50%; background-color: #9c27b0; margin-right: 10px; position: absolute; left: -20px; top: 8px;"></span>
    <strong>충분한 수분 섭취</strong> – 영양소 흡수를 돕고 관절 건강을 유지하기 위해 물을 충분히 마시는 것이 중요합니다.
  </li>
</ul>

<!-- 여섯 번째 섹션 -->
<h2 id="lifestyle-tips" style="font-size: 1.8rem; font-weight: 700; color: #4a148c; margin-top: 1.5em; margin-bottom: 0.8em; line-height: 1.2; border-left: 5px solid #9c27b0; padding-left: 0.8em; font-family: 'Noto Sans KR', sans-serif;">
관절 건강을 위한 생활 습관
</h2>

<p style="font-size: 1rem; line-height: 1.8; margin-bottom: 1.2em; color: #333; font-family: 'Noto Sans KR', sans-serif;">
관절 건강은 단순히 영양제만으로 해결되는 것이 아닙니다. 생활 속에서 실천할 수 있는 작은 습관들이 오랫동안 관절을 건강하게 유지하는 데 큰 도움이 됩니다.
</p>

<!-- 생활 습관 리스트 -->
<ul style="margin-bottom: 1.2em; padding-left: 2em; list-style-type: none; color: #333; font-family: 'Noto Sans KR', sans-serif;">
  <li style="margin-bottom: 0.5em; position: relative;">
    <span style="display: inline-block; width: 8px; height: 8px; border-radius: 50%; background-color: #9c27b0; margin-right: 10px; position: absolute; left: -20px; top: 8px;"></span>
    <strong>적절한 운동</strong> – 무리한 운동보다 관절에 부담이 적은 수영, 걷기, 요가 등을 추천합니다.
  </li>
  <li style="margin-bottom: 0.5em; position: relative;">
    <span style="display: inline-block; width: 8px; height: 8px; border-radius: 50%; background-color: #9c27b0; margin-right: 10px; position: absolute; left: -20px; top: 8px;"></span>
    <strong>체중 관리</strong> – 무릎 관절에 부담을 주지 않도록 정상 체중을 유지하는 것이 중요합니다.
  </li>
  <li style="margin-bottom: 0.5em; position: relative;">
    <span style="display: inline-block; width: 8px; height: 8px; border-radius: 50%; background-color: #9c27b0; margin-right: 10px; position: absolute; left: -20px; top: 8px;"></span>
    <strong>바른 자세 유지</strong> – 장시간 앉아있을 때 올바른 자세를 유지하고 자주 스트레칭을 해 주세요.
  </li>
</ul>

<!-- FAQ 섹션 -->
<h2 style="font-size: 1.8rem; font-weight: 700; color: #4a148c; margin-top: 1.5em; margin-bottom: 0.8em; line-height: 1.2; border-left: 5px solid #9c27b0; padding-left: 0.8em; font-family: 'Noto Sans KR', sans-serif;">
자주 묻는 질문 (FAQ)
</h2>

<!-- 질문 1 -->
<div style="margin: 1.5em 0; font-family: 'Noto Sans KR', sans-serif;">
  <div style="font-weight: 700; color: #9c27b0; margin-bottom: 0.5em; font-size: 1.1rem;">
    관절연골영양제를 얼마나 오래 복용해야 하나요?
  </div>
  <div style="margin-left: 20px;">
    <p style="margin: 0; line-height: 1.6;">
      일반적으로 최소 3~6개월 이상 꾸준히 복용해야 효과를 느낄 수 있습니다. 연골 건강은 시간이 지나면서 서서히 개선되므로, 단기간 복용 후 효과가 없다고 중단하지 않는 것이 중요합니다.
    </p>
  </div>
</div>

<!-- 질문 2 -->
<div style="margin: 1.5em 0; font-family: 'Noto Sans KR', sans-serif;">
  <div style="font-weight: 700; color: #9c27b0; margin-bottom: 0.5em; font-size: 1.1rem;">
    영양제만 먹으면 관절 건강이 좋아질까요?
  </div>
  <div style="margin-left: 20px;">
    <p style="margin: 0; line-height: 1.6;">
      아닙니다. 영양제는 보조적인 역할을 하며, 적절한 운동과 생활 습관이 함께 병행되어야 효과를 극대화할 수 있습니다. 무릎을 보호하는 운동과 체중 관리도 중요합니다.
    </p>
  </div>
</div>

<!-- 질문 3 -->
<div style="margin: 1.5em 0; font-family: 'Noto Sans KR', sans-serif;">
  <div style="font-weight: 700; color: #9c27b0; margin-bottom: 0.5em; font-size: 1.1rem;">
    관절염 환자도 영양제를 먹으면 도움이 될까요?
  </div>
  <div style="margin-left: 20px;">
    <p style="margin: 0; line-height: 1.6;">
      관절염이 있는 경우 영양제가 도움이 될 수 있지만, 병원에서 정식으로 처방받은 치료법과 병행하는 것이 좋습니다. 특히 심한 관절염의 경우 단순 영양제만으로는 큰 효과를 보기 어렵습니다.
    </p>
  </div>
</div>

<!-- 질문 4 -->
<div style="margin: 1.5em 0; font-family: 'Noto Sans KR', sans-serif;">
  <div style="font-weight: 700; color: #9c27b0; margin-bottom: 0.5em; font-size: 1.1rem;">
    글루코사민과 콘드로이틴을 함께 섭취해도 되나요?
  </div>
  <div style="margin-left: 20px;">
    <p style="margin: 0; line-height: 1.6;">
      네, 함께 섭취해도 괜찮습니다. 두 성분은 관절 건강을 돕는 역할을 하며, 함께 섭취하면 더 좋은 시너지 효과를 얻을 수 있습니다.
    </p>
  </div>
</div>

<!-- 질문 5 -->
<div style="margin: 1.5em 0; font-family: 'Noto Sans KR', sans-serif;">
  <div style="font-weight: 700; color: #9c27b0; margin-bottom: 0.5em; font-size: 1.1rem;">
    임산부나 수유 중인 여성도 복용할 수 있나요?
  </div>
  <div style="margin-left: 20px;">
    <p style="margin: 0; line-height: 1.6;">
      임산부나 수유 중인 여성은 복용 전 반드시 의사와 상담하는 것이 좋습니다. 일부 성분은 태아나 아기에게 영향을 미칠 수 있으므로 신중한 접근이 필요합니다.
    </p>
  </div>
</div>

<!-- 질문 6 -->
<div style="margin: 1.5em 0; font-family: 'Noto Sans KR', sans-serif;">
  <div style="font-weight: 700; color: #9c27b0; margin-bottom: 0.5em; font-size: 1.1rem;">
    영양제를 복용하면 부작용이 있을 수도 있나요?
  </div>
  <div style="margin-left: 20px;">
    <p style="margin: 0; line-height: 1.6;">
      드물지만 일부 사람들에게서 소화 불량, 속쓰림, 알러지 반응 등의 부작용이 나타날 수 있습니다. 만약 이상 반응이 발생하면 즉시 복용을 중단하고 전문의와 상담하세요.
    </p>
  </div>
</div>

<!-- 마무리 -->
<p style="font-size: 1rem; line-height: 1.8; margin-bottom: 1.2em; color: #333; font-family: 'Noto Sans KR', sans-serif;">
건강한 관절을 유지하는 것은 단순한 선택이 아니라, 삶의 질을 결정하는 중요한 요소입니다. 꾸준한 관리와 올바른 영양제 선택, 그리고 건강한 생활 습관이 함께한다면 오랫동안 관절 건강을 유지할 수 있습니다.  
오늘 소개해 드린 정보를 바탕으로 본인에게 가장 적합한 관절연골영양제를 선택해 보세요!  
혹시 더 궁금한 점이 있거나 경험을 나누고 싶다면 댓글로 공유해 주세요. 여러분의 의견이 더 많은 사람들에게 도움이 될 수 있습니다. 😊
</p>

<!-- 태그 -->
<p style="font-size: 0.9rem; color: #6a1b9a; font-weight: 500; font-family: 'Noto Sans KR', sans-serif;">
관절건강, 연골영양제, 무릎보호, 건강한삶, 관절영양제추천, 관절염예방, 글루코사민, MSM효과, 연골보호, 생활건강
</p>





SyntaxError: invalid syntax (3194852768.py, line 1)

In [8]:
markdown_str='# 관절연골영양제의 효과와 선택 가이드\n\n관절 건강은 우리의 일상생활에 큰 영향을 미칩니다. 특히 나이가 들면서 관절 통증이나 염증이 증가하는 경우가 많아, 이를 예방하고 관리하기 위한 방법이 필요합니다. 관절 연골 영양제는 이러한 문제를 해결하는 데 도움을 줄 수 있는 중요한 선택지입니다. 이 블로그에서는 관절 연골 영양제의 효과, 종류, 선택 시 고려사항 등을 다루어, 독자들이 올바른 결정을 내릴 수 있도록 안내하고자 합니다. \n\n## 요약\n\n관절 연골 영양제를 선택할 때는 여러 요소를 고려해야 합니다. 아래는 주요 성분과 그 효능을 비교한 표입니다.\n\n| 성분               | 효능                                   | 가격대       |\n|--------------------|----------------------------------------|--------------|\n| 글루코사민         | 관절액 생성 촉진, 연골 재생 도움      | 약 2만원대   |\n| 콘드로이틴         | 연골 수분 유지, 통증 완화            | 1만원대~3만원대 |\n| MSM                | 염증 감소, 관절 유연성 증가           | 약 1만원대   |\n| 오메가-3           | 항염 효과, 관절 경직 완화            | 약 2만원대   |\n| 보스웰리아         | 관절 염증 감소, 연골 보호            | 가격 다양    |\n\n이 표를 참고하여 자신의 건강 상태에 맞는 영양제를 선택하는 것이 중요합니다. 전문가의 상담을 통해 적절한 제품을 선택하고, 꾸준한 복용과 함께 건강한 생활습관을 유지하는 것이 관절 건강을 지키는 데 큰 도움이 될 것입니다.\n\n## 관절 연골 영양제의 효과\n\n관절 연골 영양제는 관절 건강을 유지하고 통증을 완화하는 데 도움을 줄 수 있는 다양한 성분을 포함하고 있습니다. 특히, 글루코사민과 콘드로이틴은 연골 구성에 중요한 역할을 하며, 연구에 따르면 이들 성분이 관절 통증을 줄이고 기능을 개선하는 데 효과적이라는 결과가 있습니다[1][4].\n\nUC-Ⅱ(비변형 2형 콜라겐)와 보스웰리아 같은 성분도 주목받고 있습니다. UC-Ⅱ를 하루 40mg 복용한 참가자들은 관절 가동 범위가 증가하고 통증이 감소한 것으로 나타났습니다[1]. 보스웰리아는 관절 염증을 줄이고 연골 보호에 도움을 주는 것으로 알려져 있습니다[1][2].\n\n또한, 오메가-3 지방산은 항염증 효과가 있어 관절의 경직과 압통을 완화하는 데 기여할 수 있습니다[6][7]. 이러한 성분들은 관절 건강을 지키기 위한 중요한 요소로, 개인의 건강 상태에 따라 적절한 영양제를 선택하는 것이 중요합니다[8].\n\n### Sources\n[1] 관절 영양제의 불편한 진실! 비싼 게 무조건 좋은 걸까? 연구자료와 실제 사례: https://ohmymento.tistory.com/129  \n[2] 연구 기반의 관절 통증을 위한 관절 영양제 9가지: https://science11.tistory.com/entry/연구-기반의-관절-통증을-위한-관절-영양제-9가지  \n[3] 관절 영양제, 글루코사민과 콘드로이친 - 미주 한국일보: http://m.koreatimes.com/article/20220802/1426669  \n[4] 관절영양제 : 글루코사민, 콘드로이친. 정말 효과가 있을까?: https://mdaslan.tistory.com/167  \n[5] 관절, 연골 영양제? 관절에 좋은 영양성분 20가지 소개!: https://keepthebody.com/관절-연골-영양제-관절에-좋은-영양성분-20가지-소개/  \n[6] 관절에 좋은 영양제 Best 5 | 연골 건강: https://health.kkss.co.kr/entry/연골에-좋은-영양제  \n[7] 관절 영양제 추천! 관절 건강을 위한 완벽 가이드: https://health.ottogidream.com/entry/관절-건강의-필수품-최고의-관절-영양제-완벽-가이드-추천-영양제-Best  \n[8] 관절 건강을 위한 필수 영양제 선택 가이드: https://fnfls1007.tistory.com/entry/관절-건강을-위한-필수-영양제-선택-가이드-올바른-선택의-길잡이  \n\n## 관절 연골 영양제 종류 소개\n\n관절 건강을 위해 다양한 영양제가 존재하며, 각 성분은 고유의 효능을 가지고 있습니다. 아래는 주요 관절 연골 영양제의 종류와 그 효능입니다.\n\n1. **글루코사민 (Glucosamine)**: 관절액 생성을 촉진하여 관절의 윤활성을 높이고, 연골 재생에 도움을 줍니다[1][2].\n\n2. **콘드로이틴 (Chondroitin)**: 연골의 수분을 유지하고, 손상을 예방하여 관절 통증을 완화하는 데 효과적입니다[1][3].\n\n3. **MSM (Methylsulfonylmethane)**: 염증을 줄이고 관절 통증을 완화하는 데 도움을 주며, 관절의 유연성을 증가시킵니다[4][5].\n\n4. **오메가-3 지방산**: 항염증 효과가 있어 관절의 경직과 통증을 완화하는 데 기여합니다[1][6].\n\n5. **보스웰리아 (Boswellia)**: 관절 염증을 감소시키고 연골 보호에 도움을 주는 성분입니다[7][8].\n\n6. **히알루론산 (Hyaluronic Acid)**: 관절의 윤활성을 높이고, 연골을 보호하는 역할을 합니다[9][10].\n\n이러한 성분들은 각각의 특성과 효능을 가지고 있어, 개인의 건강 상태에 맞춰 선택하는 것이 중요합니다.\n\n### Sources\n[1] 관절, 연골 영양제? 관절에 좋은 영양성분 20가지 소개! - https://keepthebody.com/관절-연골-영양제-관절에-좋은-영양성분-20가지-소개/  \n[2] 관절 영양제 연골 재생 효과 최적의 성분, 추천 제품! - https://a.sdbl1.com/관절-영양제-연골-재생-효과-추천-성분-비교/  \n[3] 관절 영양제 추천| 종류별 비교 & 실제 후기 | 관절 건강, 연골 재생, 효과 비교 - https://ceiahn.tistory.com/entry/관절-연골-영양제-추천-종류별-비교-실제-후기-관절-건강-연골-재생-효과-비교  \n[4] 글루코사민, 콘드로이틴, Msm: 효능, 부작용, 섭취량 총정리! - https://djdjskso.tistory.com/entry/글루코사민-콘드로이틴-MSM-효능-부작용-섭취량-총정리  \n[5] 관절 영양제 추천 총정리 (무릎, 어깨, 손목, 허리) : 네이버 블로그 - https://m.blog.naver.com/fn3995/223155062505  \n[6] 관절 영양제의 불편한 진실! 비싼 게 무조건 좋은 걸까? 연구자료와 실제 사례 - https://ohmymento.tistory.com/129  \n[7] 관절 영양제 추천| 종류별 비교 & 실제 후기 | 관절 건강, 연골 재생, 효과 비교 - https://ceiahn.tistory.com/entry/관절-연골-영양제-추천-종류별-비교-실제-후기-관절-건강-연골-재생-효과-비교  \n[8] 관절 건강을 위한 최고의 영양제 추천 및 가이드 - https://alryeo-dream.tistory.com/entry/관절-건강을-위한-최고의-영양제-추천-및-가이드  \n[9] 관절 영양제 추천| 종류별 비교 & 실제 후기 | 관절 건강, 연골 재생, 효과 비교 - https://ceiahn.tistory.com/entry/관절-연골-영양제-추천-종류별-비교-실제-후기-관절-건강-연골-재생-효과-비교  \n[10] 관절 건강을 위한 필수 영양제 선택 가이드, 올바른 선택의 길잡이 - https://fnfls1007.tistory.com/entry/관절-건강을-위한-필수-영양제-선택-가이드-올바른-선택의-길잡이\n\n## 관절 연골 영양제 선택 시 고려사항\n\n관절 영양제를 선택할 때는 몇 가지 중요한 요소를 고려해야 합니다. 첫째, 제품의 성분을 확인하는 것이 필수적입니다. 글루코사민, 콘드로이틴, MSM(메틸설포닐메탄) 등은 관절 건강에 도움을 주는 주요 성분입니다. 이들 성분은 연골 보호와 통증 완화에 효과적입니다[1][2].\n\n둘째, 개인의 건강 상태와 필요에 맞는 제품을 선택해야 합니다. 예를 들어, 관절 통증이 심한 경우에는 고용량의 글루코사민이 포함된 제품이 좋습니다. 반면, 예방 차원에서는 MSM이나 오메가-3가 포함된 제품이 적합할 수 있습니다[3][4].\n\n셋째, 복용 방법과 시간을 고려해야 합니다. 관절 영양제는 식사와 함께 복용하는 것이 흡수율을 높이는 데 도움이 됩니다. 또한, 최소 3개월 이상 꾸준히 복용해야 효과를 볼 수 있습니다[5][6]. 마지막으로, 신뢰할 수 있는 브랜드의 제품을 선택하는 것이 중요합니다. \n\n이러한 요소들을 종합적으로 고려하여 자신에게 맞는 관절 영양제를 선택하는 것이 관절 건강을 유지하는 데 큰 도움이 될 것입니다.\n\n### Sources\n[1] 관절 건강을 위한 최고의 영양제 추천 및 가이드: https://alryeo-dream.tistory.com/entry/관절-건강을-위한-최고의-영양제-추천-및-가이드  \n[2] 관절 영양제, 종류와 효능, 부작용 및 부작용 줄이는 법: https://m.blog.naver.com/jsd7616/223686110083  \n[3] 관절 영양제, 어떻게 현명하게 선택할까? 구매 가이드 공개: https://allmoments.tistory.com/entry/관절-영양제-어떻게-현명하게-선택할까-구매-가이드-공개  \n[4] 관절 영양제 효능 및 올바른 선택 방법 — 건강보감: https://health-guide.tistory.com/entry/관절-영양제-효능-및-올바른-선택-방법  \n[5] 관절 영양제 복용 기간과 주의사항, 전문가의 조언: https://todayfortunes.tistory.com/386  \n[6] 관절 영양제 복용 시 주의사항과 부작용, 꼭 알아두세요: https://todayfortunes.tistory.com/380  \n\n## 관절 연골 영양제 비교표\n\n관절 건강을 위한 영양제는 다양한 성분으로 구성되어 있으며, 각 제품마다 효능과 가격이 다릅니다. 아래는 주요 영양제의 성분, 효능, 가격을 비교한 표입니다.\n\n1. **글루코사민**: 관절 연골의 주요 구성 성분으로, 연골 보호 및 관절염 증상 완화에 도움을 줍니다. 가격대는 약 2만원대입니다[1][2].\n\n2. **콘드로이틴**: 연골의 수분 유지 및 충격 흡수에 기여하며, 관절 통증 완화에 효과적입니다. 가격은 1만원대에서 3만원대까지 다양합니다[3][4].\n\n3. **MSM (Methylsulfonylmethane)**: 항염 및 통증 완화에 도움을 주며, 관절 건강뿐만 아니라 피부와 모발 건강에도 긍정적인 영향을 미칩니다. 가격은 약 1만원대입니다[5][6].\n\n4. **오메가-3**: 염증을 줄이고 관절의 유연성을 증가시키는 데 도움을 줍니다. 가격은 2만원대입니다[7][8].\n\n5. **비타민 D**: 칼슘 흡수를 촉진하여 관절 건강에 중요한 역할을 합니다. 가격은 1만원대입니다[9].\n\n이 표를 참고하여 자신의 건강 상태에 맞는 영양제를 선택하는 것이 중요합니다. 전문가의 상담을 통해 적절한 제품을 선택하는 것이 좋습니다.\n\n### Sources\n[1] 관절영양제 비교, 성분별 효능 및 섭취방법 확인 - 붙잡고도: https://see.anvios.com/관절영양제-비교-성분별-효능-및-섭취방법-확인/  \n[2] 관절 영양제 완벽 비교: 성분, 효과, 가격: https://myblog1209.tistory.com/entry/관절-영양제-완벽-비교-성분-효과-가격  \n[3] 관절 영양제 추천 및 효능과 효과 분석 정리: https://m.blog.naver.com/want13371/223569529035  \n[4] 관절 영양제 Msm 효능 5가지 및 부작용, 하루 권장량: https://health-research-life.com/msm/  \n[5] 관절, 연골 영양제? 관절에 좋은 영양성분 20가지 소개! - keepTheBody: https://keepthebody.com/관절-연골-영양제-관절에-좋은-영양성분-20가지-소개/  \n[6] 관절 영양제 효과, 부작용, 종류 및 추천 제품: https://athatbuilding.com/관절-영양제/  \n[7] 관절 영양제 추천 총정리 (무릎, 어깨, 손목, 허리): https://m.blog.naver.com/fn3995/223155062505  \n[8] 관절 영양제 연골 재생 효과 최적의 성분, 추천 제품! msm 보스웰리아 등 8개 성분 비교: https://a.sdbl1.com/관절-영양제-연골-재생-효과-추천-성분-비교/  \n[9] 관절 영양제 난각막 Nem 효능 성분 부작용 먹으면 안 되는 사람: https://seekandfind.tistory.com/entry/관절영양제-난각막-NEM-효능-성분-부작용-먹으면-안-되는-사람\n\n## 관절 연골 영양제 복용 팁\n\n관절 영양제를 효과적으로 복용하기 위해서는 몇 가지 중요한 팁을 기억해야 합니다.\n\n1. **복용 시간**: 영양제는 식사 후에 복용하는 것이 좋습니다. 이는 위장 부담을 줄이고 흡수율을 높이는 데 도움이 됩니다[6][8].\n\n2. **꾸준한 복용**: 최소 3개월 이상 꾸준히 복용해야 효과를 볼 수 있습니다. 관절 건강 개선은 시간이 걸리므로 인내심을 가지고 복용하세요[7][9].\n\n3. **적정 복용량 준수**: 과다 복용은 건강에 해로울 수 있습니다. 각 영양제의 권장 복용량을 준수하고, 필요시 전문가와 상담하세요[6][10].\n\n4. **상호작용 주의**: 특정 약물과의 상호작용이 있을 수 있으므로, 복용 중인 약물이 있다면 의사와 상담하는 것이 중요합니다[6][10].\n\n5. **운동과 병행**: 영양제 복용과 함께 규칙적인 운동과 스트레칭을 병행하면 관절 건강 유지에 큰 도움이 됩니다[5][9].\n\n이러한 팁을 통해 관절 영양제를 보다 효과적으로 활용할 수 있습니다.\n\n### Sources\n[1] MSM 관절 영양제 효과와 부작용 완벽 가이드: https://wellnessbutterfly0806.tistory.com/entry/MSM-관절-영양제-효과와-부작용-완벽-가이드  \n[2] 관절 영양제 복용 시 주의사항과 부작용, 꼭 알아두세요: https://todayfortunes.tistory.com/380  \n[3] 관절 영양제 복용 기간과 주의사항, 전문가의 조언: https://todayfortunes.tistory.com/386  \n[4] 관절 건강을 위한 최고의 영양제 추천 및 가이드: https://alryeo-dream.tistory.com/entry/관절-건강을-위한-최고의-영양제-추천-및-가이드  \n[5] 관절 통증 치료에 효과 있는 영양제 치료 성분 특징 복용 팁 주의사항: https://gre-en.tistory.com/entry/관절-통증-치료에-효과-있는-영양제-치료-성분-특징-복용-팁-주의사항  \n[6] 관절 영양제 복용시간 이거 모르면 말짱도루묵: https://blog.naver.com/PostView.naver?blogId=greencross2&logNo=223498175767  \n[7] 관절 건강을 위한 영양제 추천과 효과적인 섭취 방법: https://rabbitone.tistory.com/entry/관절-건강을-위한-영양제-추천과-효과적인-섭취-방법  \n[8] 관절 영양제의 부작용 및 주의 사항: https://start.nopressmk.com/entry/관절-영양제의-부작용-및-주의-사항  \n[9] 관절염 극복, 영양제 선택 완벽 가이드: 5가지 주요 성분과 효과적인 복용법: https://hiakcl.tistory.com/entry/관절염-극복-영양제-선택-완벽-가이드-5가지-주요-성분과-효과적인-복용법  \n[10] 관절 건강을 위한 필수 영양제 선택 가이드, 올바른 선택의 길잡이: https://fnfls1007.tistory.com/entry/관절-건강을-위한-필수-영양제-선택-가이드-올바른-선택의-길잡이  \n\n# 관절연골영양제의 효과와 선택 가이드\n\n관절 건강은 우리의 일상생활에 큰 영향을 미칩니다. 관절연골영양제는 이러한 건강을 유지하고 통증을 완화하는 데 도움을 줄 수 있는 중요한 요소입니다. 글루코사민, 콘드로이틴, 오메가-3 지방산 등 다양한 성분이 포함되어 있으며, 이들은 관절의 윤활성을 높이고 염증을 줄이는 데 기여합니다. 이 블로그에서는 관절연골영양제의 효과, 종류, 선택 시 고려사항 등을 자세히 살펴보겠습니다.\n\n## Q&A\n\n관절연골영양제에 대한 독자들의 자주 묻는 질문과 그에 대한 답변을 통해 정보의 깊이를 더해보겠습니다. 관절 건강을 위한 영양제 선택에 있어 궁금한 점이 있다면, 아래의 질문과 답변을 참고하시기 바랍니다.\n\n# 관절연골영양제의 효과와 선택 가이드\n\n## 결론\n\n관절 연골 영양제는 관절 건강을 유지하고 통증을 완화하는 데 중요한 역할을 합니다. 글루코사민, 콘드로이틴, MSM, 오메가-3, 보스웰리아 등 다양한 성분이 각각의 효능을 가지고 있어, 개인의 건강 상태에 맞춰 적절한 제품을 선택하는 것이 중요합니다. 아래는 주요 성분과 그 효능을 정리한 표입니다.\n\n| 성분          | 효능                                   |\n|---------------|----------------------------------------|\n| 글루코사민    | 관절액 생성 촉진, 연골 재생 도움      |\n| 콘드로이틴    | 연골 수분 유지, 통증 완화             |\n| MSM           | 염증 감소, 관절 유연성 증가           |\n| 오메가-3      | 항염 효과, 관절 경직 완화             |\n| 보스웰리아    | 관절 염증 감소, 연골 보호             |\n\n관절 영양제를 선택할 때는 성분 확인, 개인 건강 상태 고려, 복용 방법 및 신뢰할 수 있는 브랜드 선택이 필수적입니다. 꾸준한 복용과 함께 규칙적인 운동을 병행하면 더욱 효과적인 관절 건강 유지가 가능합니다.'

In [7]:
markdown_str = events[-1]['compile_final_blog']['final_blog']
from IPython.display import Markdown
Markdown(markdown_str)

NameError: name 'events' is not defined

In [9]:
# src/markdown_to_html_converter.py
"""
마크다운 문서를 HTML로 변환하는 시스템.
LangChain과 LangGraph를 활용하여 문서 내용을 분석하고 스타일이 적용된 HTML로 변환합니다.
(개선 버전)
"""

import os
import re
import json
from typing import Dict, List, Optional, Any
from enum import Enum
from dotenv import load_dotenv
from typing_extensions import TypedDict

# 마크다운 파싱 라이브러리
import mistune
from mistune.plugins import TablePlugin
from langchain_core.messages import HumanMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, END

# pydantic (Function Calling을 위한 모델)
try:
    from pydantic import BaseModel, Field
except ImportError:
    from pydantic.v1 import BaseModel, Field

load_dotenv()

#################################
# 1) 테마, 스타일, DocumentState #
#################################
class Theme(str, Enum):
    PURPLE = "purple"
    GREEN = "green"
    BLUE = "blue"
    ORANGE = "orange"

THEME_STYLES = {
    Theme.PURPLE: {
        "h1": "font-size: 2.2rem; font-weight: 700; color: #4a148c; margin-top: 1.5em; margin-bottom: 0.8em; line-height: 1.2; border-bottom: 3px solid #7b1fa2; padding-bottom: 0.3em; font-family: 'Noto Sans KR', sans-serif;",
        "h2": "font-size: 1.8rem; font-weight: 700; color: #4a148c; margin-top: 1.5em; margin-bottom: 0.8em; line-height: 1.2; border-left: 5px solid #9c27b0; padding-left: 0.8em; font-family: 'Noto Sans KR', sans-serif;",
        "h3": "font-size: 1.5rem; font-weight: 700; color: #ffffff; background-color: #6a1b9a; padding: 0.5em 1em; border-radius: 4px; width: 100%; box-sizing: border-box; margin-top: 1.5em; margin-bottom: 0.8em; line-height: 1.2; font-family: 'Noto Sans KR', sans-serif;",
        "h4": "font-size: 1.2rem; font-weight: 700; color: #8e24aa; margin-top: 1.5em; margin-bottom: 0.8em; line-height: 1.2; font-style: italic; font-family: 'Noto Sans KR', sans-serif;",
        "p": "font-size: 1rem; line-height: 1.8; margin-bottom: 1.2em; color: #333; font-family: 'Noto Sans KR', sans-serif;",
        "strong": "font-weight: 700; color: #6a1b9a; font-family: 'Noto Sans KR', sans-serif;",
        "b": "font-weight: 700; color: #8e24aa; font-family: 'Noto Sans KR', sans-serif;",
        "blockquote": "border-left: 4px solid #9c27b0; padding: 0.5em 1em; margin: 1.5em 0; font-style: italic; color: #6a1b9a; background-color: #f9f2ff; font-family: 'Noto Sans KR', sans-serif;",
        "table_header": "border: 1px solid #e1bee7; padding: 0.5em; text-align: left; background-color: #9c27b0; color: white; font-weight: 700;",
        "table_cell": "border: 1px solid #e1bee7; padding: 0.5em; text-align: left;",
        "toc": "background: linear-gradient(135deg, #f5f0ff, #f0e6ff); border-radius: 8px; padding: 20px; margin: 20px 0; box-shadow: 0 4px 10px rgba(106, 27, 154, 0.08); font-family: 'Noto Sans KR', sans-serif; border-left: 4px solid #9c27b0;",
        "toc_title": "font-weight: 700; color: #4a148c; font-size: 18px; margin: 0;",
        "toc_link": "color: #6a1b9a; text-decoration: none; font-weight: 500;",
        "list_marker": "display: inline-block; width: 8px; height: 8px; border-radius: 50%; background-color: #9c27b0; margin-right: 10px; position: absolute; left: -20px; top: 8px;",
        "accent_color": "#9c27b0",
        "accent_bg": "#f5f0ff"
    },
    Theme.GREEN: {
        "h1": "font-size: 2.2rem; font-weight: 700; color: #1b5e20; margin-top: 1.5em; margin-bottom: 0.8em; line-height: 1.2; border-bottom: 3px solid #388e3c; padding-bottom: 0.3em; font-family: 'Noto Sans KR', sans-serif;",
        "h2": "font-size: 1.8rem; font-weight: 700; color: #1b5e20; margin-top: 1.5em; margin-bottom: 0.8em; line-height: 1.2; border-left: 5px solid #4caf50; padding-left: 0.8em; font-family: 'Noto Sans KR', sans-serif;",
        "h3": "font-size: 1.5rem; font-weight: 700; color: #ffffff; background-color: #2e7d32; padding: 0.5em 1em; border-radius: 4px; width: 100%; box-sizing: border-box; margin-top: 1.5em; margin-bottom: 0.8em; line-height: 1.2; font-family: 'Noto Sans KR', sans-serif;",
        "h4": "font-size: 1.2rem; font-weight: 700; color: #43a047; margin-top: 1.5em; margin-bottom: 0.8em; line-height: 1.2; font-style: italic; font-family: 'Noto Sans KR', sans-serif;",
        "p": "font-size: 1rem; line-height: 1.8; margin-bottom: 1.2em; color: #333; font-family: 'Noto Sans KR', sans-serif;",
        "strong": "font-weight: 700; color: #2e7d32; font-family: 'Noto Sans KR', sans-serif;",
        "b": "font-weight: 700; color: #43a047; font-family: 'Noto Sans KR', sans-serif;",
        "blockquote": "border-left: 4px solid #4caf50; padding: 0.5em 1em; margin: 1.5em 0; font-style: italic; color: #2e7d32; background-color: #e8f5e9; font-family: 'Noto Sans KR', sans-serif;",
        "table_header": "border: 1px solid #c8e6c9; padding: 0.5em; text-align: left; background-color: #4caf50; color: white; font-weight: 700;",
        "table_cell": "border: 1px solid #c8e6c9; padding: 0.5em; text-align: left;",
        "toc": "background: linear-gradient(135deg, #e8f5e9, #c8e6c9); border-radius: 8px; padding: 20px; margin: 20px 0; box-shadow: 0 4px 10px rgba(46, 125, 50, 0.08); font-family: 'Noto Sans KR', sans-serif; border-left: 4px solid #4caf50;",
        "toc_title": "font-weight: 700; color: #1b5e20; font-size: 18px; margin: 0;",
        "toc_link": "color: #2e7d32; text-decoration: none; font-weight: 500;",
        "list_marker": "display: inline-block; width: 8px; height: 8px; border-radius: 50%; background-color: #4caf50; margin-right: 10px; position: absolute; left: -20px; top: 8px;",
        "accent_color": "#4caf50",
        "accent_bg": "#e8f5e9"
    },
}

class DocumentState:
    def __init__(self, markdown_text: str, theme: Optional[Theme] = None):
        self.markdown_text = markdown_text
        self.theme = theme if theme else Theme.PURPLE
        self.html_output = ""
        self.document_structure: Dict[str, Any] = {}
        self.sections: List[Dict[str, Any]] = []
        self.errors: List[str] = []
        self.processing_complete = False

    def to_dict(self) -> Dict[str, Any]:
        return {
            "markdown_text": self.markdown_text,
            "theme": self.theme,
            "html_output": self.html_output,
            "document_structure": self.document_structure,
            "sections": self.sections,
            "errors": self.errors,
            "processing_complete": self.processing_complete,
        }

    @classmethod
    def from_dict(cls, state_dict: Dict[str, Any]) -> 'DocumentState':
        obj = cls(state_dict["markdown_text"], state_dict.get("theme"))
        obj.html_output = state_dict.get("html_output", "")
        obj.document_structure = state_dict.get("document_structure", {})
        obj.sections = state_dict.get("sections", [])
        obj.errors = state_dict.get("errors", [])
        obj.processing_complete = state_dict.get("processing_complete", False)
        return obj

#########################################
# 2) LLM 초기화 + ThemeOutput (구조화 모델)#
#########################################
def get_llm(model_name: str = "gpt-4o-mini", temperature: float = 0.0) -> ChatOpenAI:
    return ChatOpenAI(
        model=model_name,
        temperature=temperature,
        api_key=os.getenv("OPENAI_API_KEY")
    )

class ThemeOutput(BaseModel):
    recommended_theme: str = Field(..., description="문서에 적합한 테마. purple, green, blue, orange 중 하나")

##################################
# 3) 로컬 파싱 (mistune) Renderer #
##################################
class CustomRenderer(mistune.HTMLRenderer):
    def __init__(self):
        super().__init__()
        self.sections: List[Dict[str, Any]] = []
        self.section_counter = 0

    def heading(self, text, level):
        self.section_counter += 1
        section_id = f"section-{self.section_counter}"
        self.sections.append({
            "type": "subtitle" if level > 1 else "title",
            "content": text,
            "level": level,
            "section_id": section_id
        })
        return f'<h{level} id="{section_id}">{text}</h{level}>\n'

    def paragraph(self, text):
        self.section_counter += 1
        section_id = f"section-{self.section_counter}"
        self.sections.append({
            "type": "paragraph",
            "content": text,
            "level": 0,
            "section_id": section_id
        })
        return f'<p id="{section_id}">{text}</p>\n'

    def list(self, text, ordered, level, start=None):
        self.section_counter += 1
        section_id = f"section-{self.section_counter}"
        list_type = "ul" if not ordered else "ol"
        self.sections.append({
            "type": "list",
            "content": text,
            "level": level,
            "section_id": section_id
        })
        return f'<{list_type} id="{section_id}">\n{text}</{list_type}>\n'

    def table(self, header, body):
        self.section_counter += 1
        section_id = f"section-{self.section_counter}"
        self.sections.append({
            "type": "table",
            "content": header + body,
            "level": 0,
            "section_id": section_id
        })
        return f'<table id="{section_id}">\n{header}{body}</table>\n'

    def block_quote(self, text):
        self.section_counter += 1
        section_id = f"section-{self.section_counter}"
        self.sections.append({
            "type": "quote",
            "content": text,
            "level": 0,
            "section_id": section_id
        })
        return f'<blockquote id="{section_id}">{text}</blockquote>\n'

    def block_code(self, code, info=None):
        self.section_counter += 1
        section_id = f"section-{self.section_counter}"
        self.sections.append({
            "type": "code",
            "content": code,
            "level": 0,
            "section_id": section_id
        })
        language = info or ""
        return f'<pre id="{section_id}"><code class="language-{language}">{code}</code></pre>\n'

def parse_markdown_locally(markdown_text: str) -> List[Dict[str, Any]]:
    renderer = CustomRenderer()
    parser = mistune.create_markdown(renderer=renderer, plugins=[TablePlugin()])
    _ = parser(markdown_text)
    return renderer.sections

##########################################
# 4) analyze_document_structure (수정됨) #
##########################################
def analyze_document_structure(state: Dict[str, Any]) -> Dict[str, Any]:
    """
    1) 로컬 파싱으로 섹션 생성
    2) LLM으로부터 recommended_theme만 안전하게 받아온다 (structured output)
    3) 파싱 실패하면 theme=purple로 fallback
    """
    doc_state = DocumentState.from_dict(state)

    # (1) 로컬 파싱
    sections = parse_markdown_locally(doc_state.markdown_text)
    doc_state.sections = sections

    # (2) LLM으로부터 recommended_theme만 받아오기
    llm = get_llm()
    
    # structured output을 지원하는 chain 생성
    #   method="function_calling" 반드시 지정
    theme_prompt = ChatPromptTemplate.from_messages([
        ("system", """당신은 마크다운 분석 전문가입니다.
문서 내용(한국어)과 전반적 분위기를 보고, purple/green/blue/orange 중 하나를 추천하세요.

출력 예:
{
  "recommended_theme": "purple"
}

절대 다른 key를 넣지 말고 recommended_theme만 넣으세요.
"""),
        ("human", "{markdown_text}")
    ])
    
    # with_structured_output(ThemeOutput)으로 안전 파싱
    chain = theme_prompt | llm.with_structured_output(ThemeOutput, method="function_calling")
    try:
        result = chain.invoke({"markdown_text": doc_state.markdown_text})
        theme_str = result.recommended_theme.lower().strip()
        if theme_str in [t.value for t in Theme]:
            doc_state.theme = Theme(theme_str)
    except Exception as e:
        doc_state.errors.append(f"테마 분석 실패: {str(e)}")
        # fallback
        doc_state.theme = Theme.PURPLE

    return doc_state.to_dict()

#############################
# 5) 섹션별 HTML 변환 함수 #
#############################
def convert_section_to_html(state: Dict[str, Any]) -> Dict[str, Any]:
    doc_state = DocumentState.from_dict(state)
    sections = doc_state.sections
    theme = doc_state.theme
    theme_styles = THEME_STYLES.get(theme, THEME_STYLES[Theme.PURPLE])

    html_output_list = []

    for sec in sections:
        s_type = sec.get("type", "paragraph")
        s_content = sec.get("content", "")
        level = sec.get("level", 0)
        s_id = sec.get("section_id", "")

        if s_type in ["title", "subtitle"]:
            tag = f"h{level}" if 1 <= level <= 6 else "h2"
            style_key = f"h{level}" if f"h{level}" in theme_styles else "h2"
            html_piece = f'<{tag} id="{s_id}" style="{theme_styles[style_key]}">{s_content}</{tag}>'
        elif s_type == "paragraph":
            html_piece = f'<p id="{s_id}" style="{theme_styles["p"]}">{s_content}</p>'
        elif s_type == "list":
            html_piece = f'<ul id="{s_id}">{s_content}</ul>'
        elif s_type == "code":
            html_piece = f'''
<pre id="{s_id}" style="background: #f5f2ff; padding: 1em;">
<code>{s_content}</code>
</pre>
'''
        elif s_type == "table":
            html_piece = f'<table id="{s_id}" style="border-collapse: collapse; margin-bottom: 1em;">{s_content}</table>'
        elif s_type == "quote":
            html_piece = f'<blockquote id="{s_id}" style="{theme_styles["blockquote"]}">{s_content}</blockquote>'
        else:
            html_piece = f'<p id="{s_id}" style="{theme_styles["p"]}">{s_content}</p>'

        html_output_list.append(html_piece)

    doc_state.html_output = "\n".join(html_output_list)
    return doc_state.to_dict()

##########################
# 6) 목차(Toc) 항상 생성 #
##########################
def generate_toc(state: Dict[str, Any]) -> Dict[str, Any]:
    """
    FAQ, TOC는 항상 사용한다고 가정 -> has_toc가 없어도 무조건 TOC 생성
    """
    doc_state = DocumentState.from_dict(state)
    theme_styles = THEME_STYLES.get(doc_state.theme, THEME_STYLES[Theme.PURPLE])
    sections = doc_state.sections

    # h1~h3만 목차 수집
    toc_items = []
    seen_titles = set()
    for sec in sections:
        if sec["type"] in ["title", "subtitle"]:
            lvl = sec["level"] if sec["level"] else 1
            if lvl > 3:
                continue
            text = sec["content"].strip()
            if text not in seen_titles:
                toc_items.append({
                    "title": text,
                    "id": sec["section_id"],
                    "level": lvl
                })
                seen_titles.add(text)

    if not toc_items:
        return doc_state.to_dict()

    toc_html = f'<div style="{theme_styles["toc"]}">\n'
    toc_html += f'  <div style="margin-bottom: 15px;">\n'
    toc_html += f'    <h3 style="{theme_styles["toc_title"]}">목차</h3>\n'
    toc_html += f'  </div>\n'
    toc_html += f'  <div style="display: flex; flex-direction: column; gap: 10px;">\n'
    for item in toc_items:
        indent = "  " * (item["level"] - 1) if item["level"] > 1 else ""
        toc_html += f'{indent}<a href="#{item["id"]}" style="{theme_styles["toc_link"]}">{item["title"]}</a>\n'
    toc_html += f'  </div>\n'
    toc_html += f'</div>\n'

    # h1 뒤, 또는 문서 맨 앞
    html_output = doc_state.html_output
    pos_h1_end = html_output.find("</h1>")
    if pos_h1_end != -1:
        pos_h2 = html_output.find("<h2", pos_h1_end)
        if pos_h2 != -1:
            new_html = html_output[:pos_h2] + toc_html + html_output[pos_h2:]
        else:
            new_html = html_output[:pos_h1_end+5] + toc_html + html_output[pos_h1_end+5:]
        doc_state.html_output = new_html
    else:
        doc_state.html_output = toc_html + "\n" + html_output

    return doc_state.to_dict()

###########################################
# 7) 최종 검토(Conclusion 중복 등) 항상 FAQ #
###########################################
def final_review(state: Dict[str, Any]) -> Dict[str, Any]:
    """
    최종 HTML 코드 검토 (결론 중복 제거 등)
    FAQ도 무조건 포함된다고 가정 -> 별도 로직 없음
    """
    doc_state = DocumentState.from_dict(state)
    html_output = doc_state.html_output

    llm = get_llm()
    prompt = ChatPromptTemplate.from_messages([
        ("system", """당신은 HTML 코드 검토 전문가입니다.
다음 HTML 코드를 검토하고 필요한 경우 수정하세요.

확인해야 할 사항:
1. 모든 HTML 태그가 올바르게 닫혔는지
2. 모든 style 속성이 올바른 형식인지
3. id 속성이 적절하게 사용되었는지
4. 전체적인 구조가 일관되고 가독성이 좋은지
5. 'Conclusion', '결론', '마무리' 등 결론이 중복이면 하나만 남기세요.

수정한 완전한 HTML 코드만 반환하세요. 다른 설명이나 주석은 넣지 마세요.
        """),
        ("human", "{html}")
    ])
    chain = prompt | llm
    response = chain.invoke({"html": html_output})

    doc_state.html_output = response.content.strip()
    doc_state.processing_complete = True
    return doc_state.to_dict()

##########################
# 8) 워크플로우 구성/실행 #
##########################
def create_markdown_to_html_graph() -> StateGraph:
    class WorkflowState(TypedDict):
        markdown_text: str
        theme: Optional[Theme]
        html_output: str
        document_structure: Dict[str, Any]
        sections: List[Dict[str, Any]]
        errors: List[str]
        processing_complete: bool

    workflow = StateGraph(WorkflowState)
    workflow.add_node("analyze_document", analyze_document_structure)
    workflow.add_node("convert_section", convert_section_to_html)
    workflow.add_node("generate_toc", generate_toc)
    workflow.add_node("final_review", final_review)

    workflow.set_entry_point("analyze_document")
    workflow.add_edge("analyze_document", "convert_section")
    workflow.add_edge("convert_section", "generate_toc")
    workflow.add_edge("generate_toc", "final_review")
    workflow.add_edge("final_review", END)
    return workflow.compile()

def convert_markdown_to_html(markdown_text: str, theme: Optional[Theme] = None) -> str:
    doc_state = DocumentState(markdown_text, theme)
    initial_state = doc_state.to_dict()

    graph = create_markdown_to_html_graph()
    final_state = graph.invoke(initial_state)

    return final_state["html_output"]

#######################
# 9) 실행 예시 (테스트)#
#######################



In [10]:
# from src.markdown_to_html_converter import convert_markdown_to_html
html_str = convert_markdown_to_html(markdown_str)


TypeError: CustomRenderer.table() missing 1 required positional argument: 'body'

In [4]:
html_str

'```html\n<h1 id="section-1" style="font-size: 2.2rem; font-weight: 700; color: #4a148c; margin-top: 1.5em; margin-bottom: 0.8em; line-height: 1.2; border-bottom: 3px solid #7b1fa2; padding-bottom: 0.3em; font-family: \'Noto Sans KR\', sans-serif;">관절연골영양제의 효과와 선택 가이드</h1>\n<p id="section-2" style="font-size: 1rem; line-height: 1.8; margin-bottom: 1.2em; color: #333; font-family: \'Noto Sans KR\', sans-serif;">관절 건강은 우리의 일상생활에 큰 영향을 미칩니다. 특히 나이가 들면서 관절 통증이나 염증이 증가하는 경우가 많아, 이를 예방하고 관리하기 위한 방법이 필요합니다. 관절 연골 영양제는 이러한 문제를 해결하는 데 도움을 줄 수 있는 중요한 선택지입니다. 이 블로그에서는 관절 연골 영양제의 효과, 종류, 선택 시 고려사항 등을 다루어, 독자들이 올바른 결정을 내릴 수 있도록 안내하고자 합니다.</p>\n<div style="background: linear-gradient(135deg, #f5f0ff, #f0e6ff); border-radius: 8px; padding: 20px; margin: 20px 0; box-shadow: 0 4px 10px rgba(106, 27, 154, 0.08); font-family: \'Noto Sans KR\', sans-serif; border-left: 4px solid #9c27b0;">\n  <div style="margin-bottom: 15px;">\n    <h3 style="font-weight: 700; color: #4a148c; font-size: 18px; margin: 0;">

In [5]:
html_str = html_str.replace("```html","").replace("```","").replace("html","")

In [7]:

with open("html_output.html", "w", encoding="utf-8") as f:
    f.write(html_str.replace("""<?xml version="1.0" encoding="utf-8" ?>""",""))

print("다이어그램 삽입 완료! 결과는 html_output.html 에 저장되었습니다.")


다이어그램 삽입 완료! 결과는 html_output.html 에 저장되었습니다.


In [13]:
from bs4 import BeautifulSoup

def extract_sections_by_h2(html_str: str):
    """
    HTML 문자열을 파싱하여 <h2> 단위로 섹션을 구분하고,
    각 섹션의 제목(heading)과 해당 섹션에 속하는 내용을 함께 반환합니다.
    """
    soup = BeautifulSoup(html_str, 'html.parser')
    sections = []

    # 모든 <h2> 태그 찾기
    h2_tags = soup.find_all('h2')
    
    for h2 in h2_tags:
        # 현재 <h2> 태그의 텍스트 (제목)
        heading_text = h2.get_text(strip=True)
        
        # <h2> 이후의 형제 태그들을 순회하여,
        # 다음 <h2>가 나오기 전까지의 콘텐츠를 수집
        content_elements = []
        for sibling in h2.next_siblings:
            # 다음 <h2>가 나오면 현재 섹션을 끝내고 break
            if sibling.name == 'h2':
                break
            
            # 일반 Tag(예: p, h3, ul, ...) 인 경우만 HTML 문자열로 추가
            if hasattr(sibling, 'name'):
                content_elements.append(str(sibling))

        # 수집된 콘텐츠를 하나의 문자열로 합침
        section_content = "".join(content_elements)

        # 결과 리스트에 추가
        sections.append({
            "heading": heading_text,
            "content": section_content
        })

    return sections


sections = extract_sections_by_h2(html_str)
for idx, sec in enumerate(sections, start=1):
    print(f"=== Section {idx} ===")
    print(f"제목: {sec['heading']}")
    print(f"내용:\n{sec['content']}\n")


=== Section 1 ===
제목: 요약 및 결론
내용:

<p id="section-4" style="font-size: 1rem; line-height: 1.8; margin-bottom: 1.2em; color: #333; font-family: 'Noto Sans KR', sans-serif;">관절연골영양제를 선택할 때 고려해야 할 요소는 다음과 같습니다:</p>
<p id="section-5" style="font-size: 1rem; line-height: 1.8; margin-bottom: 1.2em; color: #333; font-family: 'Noto Sans KR', sans-serif;">| 요소                | 설명                                           |
|-------------------|----------------------------------------------|
| 성분 조합          | 글루코사민, 콘드로이틴, MSM 등 효과적인 성분 포함 |
| 개인 건강 상태      | 만성 질환이나 복용 중인 약물에 따라 조정 필요   |
| 임상 연구 기반 제품 | 과학적으로 검증된 제품 선택                   |
| 생활습관           | 규칙적인 운동과 균형 잡힌 식습관 병행 필요      |</p>
<p id="section-6" style="font-size: 1rem; line-height: 1.8; margin-bottom: 1.2em; color: #333; font-family: 'Noto Sans KR', sans-serif;">관절 건강을 위해서는 이러한 영양제를 꾸준히 섭취하는 것이 중요하며, 최소 3개월 이상 복용해야 효과를 느낄 수 있습니다. 올바른 선택과 함께 건강한 생활습관을 유지하여, 활기찬 일상을 즐기시기 바랍니다.</p>


=== Section 2 ===
제목: 관절연골영양제의 주요 성분 소개
내용:

<p

In [14]:
from bs4 import BeautifulSoup
import os
import uuid

# generate_diagram_from_text 함수를 임포트
# 아래 경로 및 함수는 사용자가 실제 구조에 맞춰 조정하세요.
from src.generator.text.diagram_generator import generate_diagram_from_text


def insert_diagrams_above_h2(html_str: str, diagram_output_dir: str = "./diagrams") -> str:
    """
    HTML에서 <h2> 단위로 텍스트를 추출하고, 
    각 섹션마다 generate_diagram_from_text를 이용해 
    SVG를 생성한 후 <h2> 태그 바로 위에 삽입합니다.

    Args:
        html_str (str): 기존 HTML 문자열
        diagram_output_dir (str): 다이어그램 SVG 파일을 저장할 디렉토리 경로

    Returns:
        str: SVG가 삽입된 최종 HTML 문자열
    """
    soup = BeautifulSoup(html_str, 'html.parser')

    # <h2> 태그 전체 찾기
    h2_tags = soup.find_all('h2')

    # 출력 디렉토리 준비
    os.makedirs(diagram_output_dir, exist_ok=True)

    for idx, h2 in enumerate(h2_tags, start=1):
        # 현재 <h2>부터 다음 <h2> 직전까지 텍스트를 모은다.
        section_text_fragments = []
        for sibling in h2.next_siblings:
            # 다음 h2를 만나면 해당 섹션 끝
            if sibling.name == 'h2':
                break

            # 본문 텍스트가 있을 수 있는 태그만 골라서 텍스트 추출
            if sibling.name in ['p', 'h3', 'ul', 'ol', 'li']:
                text_part = sibling.get_text(separator=' ', strip=True)
                if text_part:
                    section_text_fragments.append(text_part)

        # 섹션 전체 텍스트
        section_text = "\n".join(section_text_fragments).strip()
        if not section_text:
            # 만약 섹션 텍스트가 없다면 건너뛴다.
            continue

        # 다이어그램 SVG를 저장할 파일명
        svg_filename = f"diagram_section_{idx}.svg"
        svg_path = os.path.join(diagram_output_dir, svg_filename)

        # 다이어그램 생성
        # pixabay 키나 기타 파라미터가 필요하다면 kwargs로 추가
        generated_svg_path, diagram_info = generate_diagram_from_text(
            text=section_text,
            output_file=svg_path,
            # 예시로, 배경 색상 지정 등 가능
            background_color="#0f0f0f",
            size=500,  # 다이어그램 크기
        )

        # 생성된 SVG 파일을 읽어서 문자열로 삽입
        if os.path.exists(generated_svg_path):
            with open(generated_svg_path, 'r', encoding='utf-8') as f:
                svg_code = f.read()

            # <div> 태그 안에 SVG를 그대로 넣어서 inline embedding
            wrapper_div = soup.new_tag("div")
            # SVG 자체를 HTML로 파싱하여 wrapper_div에 추가(또는 .string = svg_code도 가능)
            wrapper_div.append(BeautifulSoup(svg_code, "html.parser"))

            # h2 태그 앞에 <div>를 삽입
            h2.insert_before(wrapper_div)

    # 수정된 soup를 문자열로 반환
    return str(soup)




updated_html = insert_diagrams_above_h2(html_str, diagram_output_dir="./diagrams_out")


2025-03-12 17:44:06,650 - src.generator.text.diagram_generator - INFO - 텍스트 분석 중...
2025-03-12 17:44:06,652 - src.generator.text.select_diagram - INFO - LLM에 텍스트 분석 요청 시작
2025-03-12 17:44:09,391 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-03-12 17:44:09,414 - src.generator.text.select_diagram - INFO - 섹션 수(4)에 따라 자동 결정된 다이어그램 유형: card
2025-03-12 17:44:09,415 - src.generator.text.select_diagram - INFO - 생성된 주제: 관절연골영양제 선택 가이드
2025-03-12 17:44:09,415 - src.generator.text.select_diagram - INFO - 생성된 섹션 수: 4개
2025-03-12 17:44:09,417 - src.generator.text.diagram_generator - INFO - LLM이 선택한 다이어그램 유형(초안): card
2025-03-12 17:44:09,419 - src.generator.text.diagram_generator - INFO - 랜덤 효과: 어두움=0.88, 블러=4.23
2025-03-12 17:44:09,421 - src.generator.text.diagram_generator - INFO - 다이어그램 생성 중: ./diagrams_out\diagram_section_1.svg
2025-03-12 17:44:09,423 - src.generator.svg.card_diagram - INFO - 최종 픽사베이 검색어: None
2025-03-12 17:44:09,430 - src

In [15]:

with open("html_output.html", "w", encoding="utf-8") as f:
    f.write(updated_html.replace("""<?xml version="1.0" encoding="utf-8" ?>""",""))

print("다이어그램 삽입 완료! 결과는 html_output.html 에 저장되었습니다.")


다이어그램 삽입 완료! 결과는 html_output.html 에 저장되었습니다.


In [31]:
updated_html

'```html\n<div style="background: linear-gradient(135deg, #f5f0ff, #f0e6ff); border-radius: 8px; padding: 20px; margin: 20px 0; box-shadow: 0 4px 10px rgba(106, 27, 154, 0.08); font-family: \'Noto Sans KR\', sans-serif; border-left: 4px solid #9c27b0;">\n<div style="margin-bottom: 15px;">\n<h3 style="font-weight: 700; color: #4a148c; font-size: 18px; margin: 0;">목차</h3>\n</div>\n<div style="display: flex; flex-direction: column; gap: 10px;">\n<a href="#importance-of-bone-density-management" style="color: #6a1b9a; text-decoration: none; font-weight: 500;">Importance of Bone Density Management</a>\n<a href="#methods-to-improve-bone-density" style="color: #6a1b9a; text-decoration: none; font-weight: 500;">Methods to Improve Bone Density</a>\n<a href="#screening-and-testing-for-bone-density" style="color: #6a1b9a; text-decoration: none; font-weight: 500;">Screening and Testing for Bone Density</a>\n<a href="#lifestyle-factors-affecting-bone-density" style="color: #6a1b9a; text-decoration: 